In [1]:
import pandas as pd
import numpy as np
import common as cm

This exercise consists of 3 parts. Finish the first part to get a mark of 3.0. The first two parts to get 4.0. Finish all exercies to get 5.0.

# Part 1: Concordance

1.1) Given are the following modes of transport: bus, bike, car, train. Each mode is characterized by 2 cost-type criteria: price and time; and 2 gain-type criteria: comfort and reliability.

Mode of transport | Time | Comfort | Price | Reliability
--- | --- | --- | --- | ---
 **bus**  | 6 | 3  | 6 | 2
 **bike** | 8 | 2  | 2 | 8
 **car**  | 2 | 10 | 9 | 7
 **train**| 3 | 5  | 5 | 6
 **by foot**| 10 | 2  | 0 | 5

In [2]:
data = {
    'mode': ['bus', 'bike', 'car', 'train', 'foot'],
    'time': [6, 8, 2, 3, 10],
    'comfort': [3, 2, 10, 5, 2],
    'price': [6, 2, 9, 5, 0],
    'reliability': [2, 8, 7, 6, 5]
}

criteria = ['time', 'comfort', 'price', 'reliability']
data = pd.DataFrame(data)

data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


1.2)  Below are the parameters, i.e., threholds, criterion-type, and weights, for each criterion,

In [3]:
parameters = {'time': {'weights': 4, 'q': 1.0, 'p': 2, 'v': 4, 'type': 'cost'},
 'comfort': {'weights': 2, 'q': 2.0, 'p': 3, 'v': 6, 'type': 'gain'},
 'price': {'weights': 3, 'q': 1.0, 'p': 3, 'v': 5, 'type': 'cost'},
 'reliability': {'weights': 1, 'q': 1.5, 'p': 3, 'v': 5, 'type': 'gain'}}
sum_weights = 10.

pd.DataFrame(parameters).T


,weights,q,p,v,type
time,4,1,2,4,cost
comfort,2,2,3,6,gain
price,3,1,3,5,cost
reliability,1,1.5,3,5,gain


1.3) Finish the below function for calculating a marginal concordance for $i$-th criterion (gain type) $c_i(g_i(a),g_i(b))$ based on q and p tresholds. 

In [4]:
def getConcordanceGain(gA, gB, q, p):
    if gA - gB >= -q:
        return 1
    elif gA - gB <= -p:
        return 0
    else:
        return (p - (gB - gA))/(p-q)
    
def getConcordanceCost(gA, gB, q, p):
    return getConcordanceGain(gB, gA, q, p)

1.4)  Calculate  comprehensive concordance  index  for  all  criteria  of  alternatives $a$ and $b$. Remember that comprehensive concordance must be divided by the sum of weights.

In [5]:
def getComprehensiveConcordance(A, B, criteria, parameters):
    a = 0
    b = 0
    for criterium in criteria:
        a += parameters[criterium]['weights'] * (getConcordanceCost(A[criterium], B[criterium], parameters[criterium]['q'], parameters[criterium]['p']) if parameters[criterium]['type'] == "cost" else getConcordanceGain(A[criterium], B[criterium], parameters[criterium]['q'], parameters[criterium]['p']))
        b += parameters[criterium]['weights']
    return a / b

1.5) Check comprehensive concordance indexes for C(bus, some transportation):

In [6]:
for alternative_id, alternative_row in data.iterrows():
    print("C({0},{1}) = ".format(0, alternative_id), getComprehensiveConcordance(data.loc[0], alternative_row, criteria, parameters))

C(0,0) =  1.0
C(0,1) =  0.6
C(0,2) =  0.3
C(0,3) =  0.5
C(0,4) =  0.6


1.6) Finish the below function for generating a concordance matrix. Use a majority_threshold as a cutting-level. For hich pairs a concordance is fulfilled?

In [7]:
def getConcordanceMatrix(data, criteria, parameters, majority_treshold=0.85):
    concordance_matrix = np.zeros((len(data),len(data)))
    for alternative_id, alternative_row in data.iterrows():
        for alternative_id_2, alternative_row_2 in data.iterrows():
            concordance_matrix[alternative_id][alternative_id_2] = 1 if getComprehensiveConcordance(alternative_row, alternative_row_2, criteria, parameters) >= majority_treshold else 0    
    return concordance_matrix

In [8]:
print(getConcordanceMatrix(data, criteria, parameters))
con = getConcordanceMatrix(data, criteria, parameters)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


# Part 2: outranking graph

2.1) Complete the function that calculates marginal discordance for  $i$-th  criterion (gain type)  $d_i(a,b)$  basing on v treshold. $d_i(a,b) = 1$ when $a$ is worse than $b$ on criterion $i$ by at least the veto threshold, zero otherwise.

In [9]:
def getDiscordanceGain(A ,B, v):
    return 1 if B - A >= v else 0

def getDiscordanceCost(A, B, v):
    return getDiscordanceGain(B, A, v)

2.2) Calculate a comprehensive discordance index.  $𝐷(a,b)=1$  if at least one criterion vetoes against $a S b$.

In [10]:
def getComprehensiveDiscordance(alternative_a, alternative_b, criteria, parameters): 
    for criterium in criteria:
        if (getDiscordanceCost(alternative_a[criterium], alternative_b[criterium], parameters[criterium]['v'])\
            if parameters[criterium]['type'] == "cost"\
                else getDiscordanceGain(alternative_a[criterium], alternative_b[criterium], parameters[criterium]['v'])):
            return 1
    return 0


2.3) Check comprehensive discordance indexes for D(bus, some transportation):

In [11]:
for alternative_id, alternative_row in data.iterrows():
    #print(data.loc[0], alternative_row)
    print("D({0},{1}) = ".format(0, alternative_id),getComprehensiveDiscordance(data.loc[0], \
                                                                                alternative_row, criteria, parameters))

D(0,0) =  0
D(0,1) =  1
D(0,2) =  1
D(0,3) =  0
D(0,4) =  1


2.4) Finish the below function for calculating a comprehensive discordance matrix.

In [12]:
def getDiscordanceMatrix(data, criteria, parameters):
    discordance_matrix = np.zeros((len(data),len(data)))
    for alternative_id, alternative_row in data.iterrows():
        for alternative_id_2, alternative_row_2 in data.iterrows():
            discordance_matrix[alternative_id][alternative_id_2] = getComprehensiveDiscordance(alternative_row, alternative_row_2, criteria, parameters)  
    return discordance_matrix

In [13]:
disc = getDiscordanceMatrix(data, criteria, parameters)
print(getDiscordanceMatrix(data, criteria, parameters))

[[0. 1. 1. 0. 1.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0.]]


2.5) Now, finish the below function for generating the outranking matrixThis method should take into account both the concordance and discordance matrices.

In [14]:
def getOutrankingMatrix(data, criteria, parameters,con,disc, majority_treshold):
    outranking_matrix = np.zeros((len(data),len(data)))
    for i in range(len(con)):
        for j in range(len(con[i])):
            outranking_matrix[i][j] = con[i][j] and not disc[i][j]
    
    return outranking_matrix

In [15]:
outranking_matrix = getOutrankingMatrix(data, criteria, parameters ,con,disc,majority_treshold=0.85)
print(con,'\n''\n',disc,'\n''\n',outranking_matrix)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]] 

 [[0. 1. 1. 0. 1.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0.]] 

 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


2.3) Change outranking matrix to adjacency list as a dictionary that every alternative $a$ has a list of alternatives that $a$ outranks. For simplicity, assume that there are no edges between the vertex and itself (a->a).

In [16]:
data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


In [17]:
def toAdjacencyList(outranking_matrix):
    adjacency_list = {i+1: [] for i in range(len(outranking_matrix))}
    for row_index, row in enumerate(outranking_matrix,1):
        for column_index, column in enumerate(row, 1):
            if row_index != column_index and column:
                adjacency_list[row_index].append(column_index)
    return adjacency_list
        

In [18]:
graph = toAdjacencyList(outranking_matrix)
# graph['34']=[1]
# del graph[3]
# del graph[4]
print(graph)

{1: [], 2: [5], 3: [], 4: [1], 5: []}


2.4) Draw outranking graph, and remove cycles (manually).

In [19]:
cm.PrintGraph(graph, filename="graph_part_2")

2.5) Which mode of transport are in the kernel?

# Part 3: Kernel

3.1) Given is the below outranking matrix

In [20]:
outranking_matrix =np.array(   [[0., 1., 0., 0., 0., 0., 1., 0.],
                                [0., 0., 1., 0., 1., 0., 0., 0.],
                                [0., 0., 0., 0., 0., 0., 0., 1.],
                                [0., 0., 0., 0., 1., 0., 1., 0.],
                                [0., 0., 0., 0., 0., 1., 0., 0.],
                                [0., 0., 0., 0., 0., 0., 1., 0.],
                                [0., 0., 0., 0., 0., 0., 0., 1.],
                                [0., 0., 0., 0., 0., 0., 0., 0.]])

In [21]:
graph = toAdjacencyList(outranking_matrix)

In [22]:
graph

{1: [2, 7], 2: [3, 5], 3: [8], 4: [5, 7], 5: [6], 6: [7], 7: [8], 8: []}

3.2) Display the outranking graph. Which alternatives belong to kernel?

In [23]:
cm.PrintGraph(graph, filename="graph_part_3")

3.3) In this exercise, you are asked to complete the function for constructing a kernel. Firstly, complete the below auxiliary function which reverses edges of the graph.

In [24]:
def getReverseGraph(graph):
    reverse_graph = {}
    for key in graph.keys():
        for element in graph[key]:
            if element not in reverse_graph.keys():
                reverse_graph[element] = []
            reverse_graph[element].append(int(key))
    return reverse_graph

3.4) Verify the correctness: compare the below reverse_graph with the above graph.

In [25]:
reverse_graph = getReverseGraph(graph)
reverse_graph

{2: [1], 7: [1, 4, 6], 3: [2], 5: [2, 4], 8: [3, 7], 6: [5]}

In [26]:
cm.PrintGraph(reverse_graph, filename="reverse_graph_part_3")

3.5) Now, complete the below function for finding a graph kernel. This algorithm should proceed in the following way: <br>
1) Find non-outranked vertices. Add them to the kernel. <br> 
2) Remove vertices found in step 1 from the graph and these vertices which are directly surpassed by them.<br>
3) Repeat (go to 1) until all vertices are removed from the graph. 

In [27]:
{1: [2, 7], 2: [3, 5], 3: [8], 4: [5, 7], 5: [6], 6: [7], 7: [8], 8: []}

def Kernel(graph):
    reverse_graph = getReverseGraph(graph)
    kernel = []
    while graph:
        elements=[]
        for graph_element in graph.keys():
            for graph_element_child in graph[graph_element]:
                if graph_element_child not in graph.keys():
                    graph[graph_element].remove(graph_element_child)
        for key in graph.keys():
            for element in graph[key]:
                elements.append(element)
        for key in graph.keys():
            if key not in set(elements):
                kernel.append(key)
        for kernel_member in kernel:
            if kernel_member in graph.keys():
                for kernel_member_child in graph[kernel_member]:
                    if kernel_member_child in graph.keys():
                        del graph[kernel_member_child]
                del graph[kernel_member]
    return kernel

In [28]:
graph = toAdjacencyList(outranking_matrix)


In [29]:
kernel = Kernel(graph)

In [30]:
print(kernel)

[1, 4, 3, 6]
